<a href="https://colab.research.google.com/github/tonystz/cuda/blob/main/poc/gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Get code

In [ ]:
!git clone https://github.com/tonystz/cuda.git

In [2]:
!python cuda/poc/raw.py

[step0]preprocess raw log: total=1153965 skip=12507 2.091172933578491



###CPU

In [3]:
!python cuda/poc/cpu.py

[step1]reload data: 3.3212413787841797 (1141458,) |S44 1141458 [b'::1-200' b'::1-200' b'::1-404' ... b'180.163.28.55-200'
 b'180.163.28.55-200' b'180.163.28.55-200']
[step2]404 data filter: 0.6432945728302002 202448
[step3]check result: 0.17370939254760742 ['216.244.66.200', '217.174.192.132', '66.249.79.234', '66.249.79.238', '66.249.79.236', '23.96.16.211', '47.101.154.149', '47.101.149.21', '38.54.24.118']


###GPU initialize

In [ ]:
!pip install pycuda # install cuda
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np

###GPU

In [ ]:
a=np.loadtxt('pre.log', delimiter=' ', dtype=np.string_,converters={0:lambda v:v+b'\0'})
print(a.shape,a.size,a)

In [21]:
print(a[0])

b'::1-200'


In [12]:
na=np.delete(a,0)
na.reshape((479,2383))

array([[b'::1-200', b'::1-404', b'::1-200', ..., b'195.242.110.253-200',
        b'110.249.202.11-200', b'110.249.201.69-200'],
       [b'110.249.201.206-200', b'207.46.13.23-200',
        b'110.249.201.14-200', ..., b'223.104.68.50-200',
        b'223.104.68.50-200', b'223.104.68.50-200'],
       [b'223.104.68.50-200', b'223.104.68.50-200', b'223.104.68.50-200',
        ..., b'125.84.82.12-200', b'125.84.82.12-200',
        b'125.84.82.12-200'],
       ...,
       [b'140.250.200.255-404', b'140.250.200.255-404',
        b'52.167.144.32-200', ..., b'180.164.68.86-200',
        b'180.164.68.86-200', b'180.164.68.86-200'],
       [b'180.164.68.86-200', b'180.164.68.86-200', b'180.164.68.86-200',
        ..., b'36.46.86.9-200', b'36.46.86.9-200', b'36.46.86.9-200'],
       [b'36.46.86.9-200', b'36.46.86.9-200', b'36.46.86.9-200', ...,
        b'180.163.28.55-200', b'180.163.28.55-200', b'180.163.28.55-200']],
      dtype='|S44')

In [15]:
s= np.array(['180.163.28.55-200\0','edfg\0','stz12345678b\0','b1\0','k2\0','hellworld3\0'], dtype=np.string_)

s=s.reshape(2,3)
print('shape:',s.shape,s.size)
print(s,s.dtype)

shape: (2, 3) 6
[[b'180.163.28.55-200' b'edfg' b'stz12345678b']
 [b'b1' b'k2' b'hellworld3']] |S18


In [28]:
%%writefile gpu.py
#!python 
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
from pycuda import gpuarray
import numpy as np

mod = SourceModule("""
    __global__ void say_hi(char*out_gpu)
    { 
      const int strLen=44;
      const int colLen=2383;
      const int rowCnt=479;
      int idx = threadIdx.x + blockIdx.x * blockDim.x;

      //printf("index: threadIdx.x=%d blockIdx.x=%d  blockDim.x=%d \\n",threadIdx.x,blockIdx.x,blockDim.x);
      printf("thread id: [%d]  get str:%s  strlen=%d \\n",idx,out_gpu+strLen+1, strLen);

      // string len is 6
      char sub[strLen+1];
      int strStartIndex=0;//string start locaion
      int rowOffset=colLen*strLen;
      if(idx > rowCnt){
        printf("[ERROR]idx=%d, rowCnt=%d, we want one thread process all one rows'data .please tune");
      }
      
      char ipAddr[16];
      int rowStartIndex=rowOffset*idx;
      for(int c=0;c<colLen;c++){
        for(int i = 0; i<strLen; i++){ 
            strStartIndex=c*strLen;
            sub[i]=out_gpu[strStartIndex+i+rowStartIndex];
            //ipAddr[i]=out_gpu[strStartIndex+i+rowStartIndex];
        }
        printf("thread[%d] rowStartIndex=%d, substring:%s strStartIndex %d\\n",idx,rowStartIndex,sub,strStartIndex);
      }
    }
    """)

say_hi = mod.get_function("say_hi")

a=np.loadtxt('pre.log', delimiter=' ', dtype=np.string_,converters={0:lambda v:v+b'\0'})
na=np.delete(a,0)
na.reshape((479,2383))
print('shape:',na.shape,na.size)
print(na,na.data)

NUM_ROW=479
NUM_COL=2383
na_gpu = gpuarray.to_gpu(na)
say_hi(na_gpu,block=(NUM_ROW,1,1),grid=(1,1,1))
print('modify:',na_gpu.get())

Overwriting gpu.py


In [29]:
!python gpu.py

Streaming output truncated to the last 5000 lines.
thread[311] rowStartIndex=32608972, substring:121.35.41.79-200 strStartIndex 104368
thread[312] rowStartIndex=32713824, substring:42.48.79.249-404 strStartIndex 104368
thread[313] rowStartIndex=32818676, substring:117.22.249.228-200 strStartIndex 104368
thread[314] rowStartIndex=32923528, substring:189.244.198.107-200 strStartIndex 104368
thread[315] rowStartIndex=33028380, substring:113.118.118.34-200 strStartIndex 104368
thread[316] rowStartIndex=33133232, substring:37.8.13.193-200 strStartIndex 104368
thread[317] rowStartIndex=33238084, substring:152.231.192.207-200 strStartIndex 104368
thread[318] rowStartIndex=33342936, substring:39.105.181.1-200 strStartIndex 104368
thread[319] rowStartIndex=33447788, substring:40.77.167.59-304 strStartIndex 104368
thread[192] rowStartIndex=20131584, substring:139.9.203.165-200 strStartIndex 104324
thread[193] rowStartIndex=20236436, substring:139.9.203.165-200 strStartIndex 104324
thread[194] ro